<a href="https://colab.research.google.com/github/strongrunner/Secure-Machine-Learning/blob/main/Encrypted_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf-encrypted
!pip install syft
! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b ryffel/4P --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets

     |████████████████████████████████| 2.7MB 5.8MB/s 
     |████████████████████████████████| 276kB 43.2MB/s 
     |████████████████████████████████| 110.5MB 78kB/s 
     |████████████████████████████████| 512kB 45.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 3.8MB 43.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=d1d0eb6fc7a444f9456707ca8b89c3eced54a860d53a15d5ee845cc0659d6d90
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=2ff49a0d93c5dc3e9a5e5a1512b1b98f52c62e31663d76fcf06d4168a891dca4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built pyyaml gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.

  Using cached https://files.pythonhosted.org/packages/bb/d9/856af84843912e2853b1b6e898ac8b802989fcf9ecf8e8445a1da263bf3b/websockets-8.1-cp36-cp36m-manylinux2010_x86_64.whl
  Found existing installation: websockets 8.1
    Uninstalling websockets-8.1:
      Successfully uninstalled websockets-8.1


## Imports and training configuration

In [ ]:
from __future__ import print_function
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torchvision import datasets, transforms
#import time
torch.__version__

'1.7.0+cu101'

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = 10
        self.lr = 0.02
        self.seed = 1
        self.log_interval = 1 # Log info at each batch
        self.precision_fractional = 3

args = Arguments()

_ = torch.manual_seed(args.seed)

Secret sharing of data in each worker

In [ ]:
import syft as sy  
hook = sy.TorchHook(torch)  
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]
def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

In [ ]:

n_train_items = 640
n_test_items = 640

def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):
 
        onehot_tensor = torch.zeros(*index_tensor.shape, 10) 
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor):
 
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transformation),
        batch_size=args.batch_size
    )
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transformation),
        batch_size=args.test_batch_size
    )
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(test_loader)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train(args, model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- now it is a private dataset
        start_time = time.time()
        
        optimizer.zero_grad()
        
        output = model(data)
        
        # loss = F.nll_loss(output, target)  <-- not possible here
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss = loss.get().float_precision()
      #      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.3f}s'.format(
       #         epoch, batch_idx * args.batch_size, len(private_train_loader) * args.batch_size,
        #        100. * batch_idx / len(private_train_loader), loss.item(), time.time() - start_time))
            

In [ ]:
def test(args, model, private_test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()
  #  print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
   #     correct.item(), len(private_test_loader)* args.test_batch_size,
    #    100. * correct.item() / (len(private_test_loader) * args.test_batch_size)))

In [ ]:
model = Net()
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
optimizer = optimizer.fix_precision() 

for epoch in range(1, args.epochs + 1):
    train(args, model, private_train_loader, optimizer, epoch)
    test(args, model, private_test_loader)

Train Epoch: 1 [0/640 (0%)]	Loss: 1.128000	Time: 6.301s
Train Epoch: 1 [64/640 (10%)]	Loss: 1.011000	Time: 4.547s
Train Epoch: 1 [128/640 (20%)]	Loss: 0.990000	Time: 4.401s
Train Epoch: 1 [192/640 (30%)]	Loss: 0.902000	Time: 5.100s
Train Epoch: 1 [256/640 (40%)]	Loss: 0.887000	Time: 4.699s
Train Epoch: 1 [320/640 (50%)]	Loss: 0.875000	Time: 4.630s
Train Epoch: 1 [384/640 (60%)]	Loss: 0.853000	Time: 4.254s
Train Epoch: 1 [448/640 (70%)]	Loss: 0.849000	Time: 4.282s
Train Epoch: 1 [512/640 (80%)]	Loss: 0.830000	Time: 4.617s
Train Epoch: 1 [576/640 (90%)]	Loss: 0.839000	Time: 4.684s

Test set: Accuracy: 221.0/640 (35%)

Train Epoch: 2 [0/640 (0%)]	Loss: 0.782000	Time: 4.239s
Train Epoch: 2 [64/640 (10%)]	Loss: 0.732000	Time: 4.516s
Train Epoch: 2 [128/640 (20%)]	Loss: 0.794000	Time: 4.403s
Train Epoch: 2 [192/640 (30%)]	Loss: 0.717000	Time: 4.787s
Train Epoch: 2 [256/640 (40%)]	Loss: 0.705000	Time: 4.560s
Train Epoch: 2 [320/640 (50%)]	Loss: 0.707000	Time: 4.364s
Train Epoch: 2 [384/640 (6